<a href="https://colab.research.google.com/github/mathewspmathew/pyspark_airquality/blob/main/project_airquality_seagmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Air Qulity (CO) and Segamentation into categories

*[Dataset link](https://archive.ics.uci.edu/dataset/360/air+quality)*

- Here I have removed null and cleaned the data.
- Then uses StandardScaler for doing Linear Regression. It predicts the target for the value of CO.
- Also tried Random Forest Regressor with the same data. compared the rmse values.

- Done segmantation to Good, Medium, Unhealthy and Danger. Used RandomForestClassifier for classification and evaluated by accuracy score

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum as spark_sum

In [2]:
spark = SparkSession.builder.appName('airquality segmentation').getOrCreate()

In [3]:
spark

In [4]:
df = spark.read.csv('/content/drive/MyDrive/datasets for colab/AirQualityUCI.csv',header = True, inferSchema=True,sep = ';')

In [5]:
df.show(3)

+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+----+----+
|      Date|    Time|CO(GT)|PT08.S1(CO)|NMHC(GT)|C6H6(GT)|PT08.S2(NMHC)|NOx(GT)|PT08.S3(NOx)|NO2(GT)|PT08.S4(NO2)|PT08.S5(O3)|   T|  RH|    AH|_c15|_c16|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+----+----+
|10/03/2004|18.00.00|   2,6|       1360|     150|    11,9|         1046|    166|        1056|    113|        1692|       1268|13,6|48,9|0,7578|NULL|NULL|
|10/03/2004|19.00.00|     2|       1292|     112|     9,4|          955|    103|        1174|     92|        1559|        972|13,3|47,7|0,7255|NULL|NULL|
|10/03/2004|20.00.00|   2,2|       1402|      88|     9,0|          939|    131|        1140|    114|        1555|       1074|11,9|54,0|0,7502|NULL|NULL|
+----------+--------+------+-----------+--------+--------+-------------+----

In [6]:
# df = df.withColumnsRenamed({'CO(GT)':'CO_GT',
#                             'PT08.S1(CO)':'PT08_S1_CO',
#                            'PT08.S2(NMHC)':'PT08_S2_NMHC_',
#                            'PT08.S3(NOx)':'PT08_S3(NOx)',
#                            'PT08.S4(NO2)':'PT08_S4(NO2)',
#                            'PT08.S5(O3)':'PT08_S5(O3)',
#                            })

In [7]:
import re

def clean_column_name(name: str) -> str:
    return re.sub(r'[^a-zA-Z0-9]', '_', name)

df = df.toDF(*[clean_column_name(c) for c in df.columns])

# this replaces all non-alphanumeric characters with underscore
# and renaming all columns also

In [8]:
df.show(2)

+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+----+----+
|      Date|    Time|CO_GT_|PT08_S1_CO_|NMHC_GT_|C6H6_GT_|PT08_S2_NMHC_|NOx_GT_|PT08_S3_NOx_|NO2_GT_|PT08_S4_NO2_|PT08_S5_O3_|   T|  RH|    AH|_c15|_c16|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+----+----+
|10/03/2004|18.00.00|   2,6|       1360|     150|    11,9|         1046|    166|        1056|    113|        1692|       1268|13,6|48,9|0,7578|NULL|NULL|
|10/03/2004|19.00.00|     2|       1292|     112|     9,4|          955|    103|        1174|     92|        1559|        972|13,3|47,7|0,7255|NULL|NULL|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+----+----+
only showing top 2 rows



In [9]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- CO_GT_: string (nullable = true)
 |-- PT08_S1_CO_: integer (nullable = true)
 |-- NMHC_GT_: integer (nullable = true)
 |-- C6H6_GT_: string (nullable = true)
 |-- PT08_S2_NMHC_: integer (nullable = true)
 |-- NOx_GT_: integer (nullable = true)
 |-- PT08_S3_NOx_: integer (nullable = true)
 |-- NO2_GT_: integer (nullable = true)
 |-- PT08_S4_NO2_: integer (nullable = true)
 |-- PT08_S5_O3_: integer (nullable = true)
 |-- T: string (nullable = true)
 |-- RH: string (nullable = true)
 |-- AH: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)



In [10]:
from pyspark.sql.functions import regexp_replace, col

numeric_cols = ["T", "RH", "CO_GT_"]
for c in numeric_cols:
    df = df.withColumn(c, regexp_replace(col(c), ",", ".").cast("double"))

df.show(5)
df.printSchema()


+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+----+----+
|      Date|    Time|CO_GT_|PT08_S1_CO_|NMHC_GT_|C6H6_GT_|PT08_S2_NMHC_|NOx_GT_|PT08_S3_NOx_|NO2_GT_|PT08_S4_NO2_|PT08_S5_O3_|   T|  RH|    AH|_c15|_c16|
+----------+--------+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+----+----+------+----+----+
|10/03/2004|18.00.00|   2.6|       1360|     150|    11,9|         1046|    166|        1056|    113|        1692|       1268|13.6|48.9|0,7578|NULL|NULL|
|10/03/2004|19.00.00|   2.0|       1292|     112|     9,4|          955|    103|        1174|     92|        1559|        972|13.3|47.7|0,7255|NULL|NULL|
|10/03/2004|20.00.00|   2.2|       1402|      88|     9,0|          939|    131|        1140|    114|        1555|       1074|11.9|54.0|0,7502|NULL|NULL|
|10/03/2004|21.00.00|   2.2|       1376|      80|     9,2|          948|    

In [11]:
null_counts = df.select([spark_sum(col(c).isNull().cast('int')).alias(c) for c in df.columns])

In [12]:
null_counts.show()

+----+----+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+---+---+---+----+----+
|Date|Time|CO_GT_|PT08_S1_CO_|NMHC_GT_|C6H6_GT_|PT08_S2_NMHC_|NOx_GT_|PT08_S3_NOx_|NO2_GT_|PT08_S4_NO2_|PT08_S5_O3_|  T| RH| AH|_c15|_c16|
+----+----+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+---+---+---+----+----+
| 114| 114|   114|        114|     114|     114|          114|    114|         114|    114|         114|        114|114|114|114|9471|9471|
+----+----+------+-----------+--------+--------+-------------+-------+------------+-------+------------+-----------+---+---+---+----+----+



In [13]:
df.count()

9471

In [14]:
df = df.drop('_c15','_c16')

In [15]:
df_needed = df.select('PT08_S1_CO_','PT08_S2_NMHC_','PT08_S3_NOx_','PT08_S4_NO2_','PT08_S5_O3_','T','RH','CO_GT_')

In [16]:
df_needed.show(2)

+-----------+-------------+------------+------------+-----------+----+----+------+
|PT08_S1_CO_|PT08_S2_NMHC_|PT08_S3_NOx_|PT08_S4_NO2_|PT08_S5_O3_|   T|  RH|CO_GT_|
+-----------+-------------+------------+------------+-----------+----+----+------+
|       1360|         1046|        1056|        1692|       1268|13.6|48.9|   2.6|
|       1292|          955|        1174|        1559|        972|13.3|47.7|   2.0|
+-----------+-------------+------------+------------+-----------+----+----+------+
only showing top 2 rows



In [17]:
df_needed.count()

9471

Selecting the needed features

In [18]:
features = ['PT08_S1_CO_','PT08_S2_NMHC_','PT08_S3_NOx_','PT08_S4_NO2_','PT08_S5_O3_','T','RH']
target = 'CO_GT_'

In [19]:
df_needed = df_needed.select([col(c).cast('double') for c in features+[target]])

In [20]:
df_needed.show(5)

+-----------+-------------+------------+------------+-----------+----+----+------+
|PT08_S1_CO_|PT08_S2_NMHC_|PT08_S3_NOx_|PT08_S4_NO2_|PT08_S5_O3_|   T|  RH|CO_GT_|
+-----------+-------------+------------+------------+-----------+----+----+------+
|     1360.0|       1046.0|      1056.0|      1692.0|     1268.0|13.6|48.9|   2.6|
|     1292.0|        955.0|      1174.0|      1559.0|      972.0|13.3|47.7|   2.0|
|     1402.0|        939.0|      1140.0|      1555.0|     1074.0|11.9|54.0|   2.2|
|     1376.0|        948.0|      1092.0|      1584.0|     1203.0|11.0|60.0|   2.2|
|     1272.0|        836.0|      1205.0|      1490.0|     1110.0|11.2|59.6|   1.6|
+-----------+-------------+------------+------------+-----------+----+----+------+
only showing top 5 rows



In [21]:
df_needed.count()

9471

In [22]:
null_counts = df_needed.select([spark_sum(col(c).isNull().cast('int')).alias(c) for c in df_needed.columns])
null_counts.show()

+-----------+-------------+------------+------------+-----------+---+---+------+
|PT08_S1_CO_|PT08_S2_NMHC_|PT08_S3_NOx_|PT08_S4_NO2_|PT08_S5_O3_|  T| RH|CO_GT_|
+-----------+-------------+------------+------------+-----------+---+---+------+
|        114|          114|         114|         114|        114|114|114|   114|
+-----------+-------------+------------+------------+-----------+---+---+------+



In [23]:
df_needed.printSchema()

root
 |-- PT08_S1_CO_: double (nullable = true)
 |-- PT08_S2_NMHC_: double (nullable = true)
 |-- PT08_S3_NOx_: double (nullable = true)
 |-- PT08_S4_NO2_: double (nullable = true)
 |-- PT08_S5_O3_: double (nullable = true)
 |-- T: double (nullable = true)
 |-- RH: double (nullable = true)
 |-- CO_GT_: double (nullable = true)



In [24]:
df_needed =df_needed.na.drop()

In [25]:
df_needed.show(5)

+-----------+-------------+------------+------------+-----------+----+----+------+
|PT08_S1_CO_|PT08_S2_NMHC_|PT08_S3_NOx_|PT08_S4_NO2_|PT08_S5_O3_|   T|  RH|CO_GT_|
+-----------+-------------+------------+------------+-----------+----+----+------+
|     1360.0|       1046.0|      1056.0|      1692.0|     1268.0|13.6|48.9|   2.6|
|     1292.0|        955.0|      1174.0|      1559.0|      972.0|13.3|47.7|   2.0|
|     1402.0|        939.0|      1140.0|      1555.0|     1074.0|11.9|54.0|   2.2|
|     1376.0|        948.0|      1092.0|      1584.0|     1203.0|11.0|60.0|   2.2|
|     1272.0|        836.0|      1205.0|      1490.0|     1110.0|11.2|59.6|   1.6|
+-----------+-------------+------------+------------+-----------+----+----+------+
only showing top 5 rows



In [26]:
df_needed.count()

9357

# MLlib Linear Regression Model (Predict CO Level)

In [95]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [28]:
features = ['PT08_S1_CO_','PT08_S2_NMHC_','PT08_S3_NOx_','PT08_S4_NO2_','PT08_S5_O3_','T','RH']
target = 'CO_GT_'

In [81]:
assembler = VectorAssembler(inputCols=features,outputCol='feature_assembled')

In [101]:
df2 = assembler.transform(df_needed)

In [102]:
df2 = df2.select('feature_assembled','CO_GT_')

In [103]:
df2.show(2)

+--------------------+------+
|   feature_assembled|CO_GT_|
+--------------------+------+
|[1360.0,1046.0,10...|   2.6|
|[1292.0,955.0,117...|   2.0|
+--------------------+------+
only showing top 2 rows



In [104]:
df2.count()

9357

Removing the -200 values because in this air quality dataset.

missing values are encoded as -200.0 instead of NaN
So Spark is treating -200.0 as real sensor readings.

In [105]:
for c in features+[target]:
    df2 = df2.filter(col(c) != -200)

df2.show(5)

+--------------------+------+
|   feature_assembled|CO_GT_|
+--------------------+------+
|[1360.0,1046.0,10...|   2.6|
|[1292.0,955.0,117...|   2.0|
|[1402.0,939.0,114...|   2.2|
|[1376.0,948.0,109...|   2.2|
|[1272.0,836.0,120...|   1.6|
+--------------------+------+
only showing top 5 rows



In [106]:
df2.count()

7344

In [107]:
scaler = StandardScaler(inputCol="feature_assembled", outputCol="scaled_features", withMean=True, withStd=True)

In [108]:
df2 = scaler.fit(df2).transform(df2).select('scaled_features','CO_GT_')

In [109]:
df2.show(2)

+--------------------+------+
|     scaled_features|CO_GT_|
+--------------------+------+
|[1.14056034018174...|   2.6|
|[0.82960558449708...|   2.0|
+--------------------+------+
only showing top 2 rows



In [110]:
df_train,df_test = df2.randomSplit([0.8,0.2],seed = 42)

In [121]:
df_train.show(2)

+--------------------+------+
|     scaled_features|CO_GT_|
+--------------------+------+
|[-2.1198917304530...|   0.3|
|[-2.1107460023446...|   0.2|
+--------------------+------+
only showing top 2 rows



In [114]:
lr = LinearRegression(featuresCol='scaled_features',labelCol='CO_GT_')

In [115]:
model = lr.fit(df_train)

In [116]:
pred = model.transform(df_test)

In [117]:
pred.show(3)

+--------------------+------+--------------------+
|     scaled_features|CO_GT_|          prediction|
+--------------------+------+--------------------+
|[-2.0833088180195...|   0.2| 0.02501156026307516|
|[-2.0192887212609...|   0.4|-0.04091277157138862|
|[-1.9781329447732...|   0.3|-0.02492848966375316|
+--------------------+------+--------------------+
only showing top 3 rows



In [118]:
evaluator = RegressionEvaluator(predictionCol='prediction',labelCol='CO_GT_',metricName='rmse')

In [119]:
rmse = evaluator.evaluate(pred)

In [120]:
rmse

0.49086031980801925

# Trying with more robust model - Random Forest Regressor

In [122]:
from pyspark.ml.regression import RandomForestRegressor

In [123]:
rf = RandomForestRegressor(featuresCol="scaled_features", labelCol="CO_GT_", numTrees=100)


In [124]:
rf_model = rf.fit(df_train)
pred = rf_model.transform(df_test)


In [126]:
evaluator = RegressionEvaluator(predictionCol='prediction',labelCol='CO_GT_',metricName='rmse')

In [129]:
rmse_rf = evaluator.evaluate(pred)

In [131]:
print('rmse of random forest : ',rmse_rf)

rmse of random forest :  0.48423802190534515


# Classification (Good, Moderate, Unhealthy)

In [41]:
from pyspark.sql.functions import when

In [42]:
df3 = df_needed.withColumn(
    'Category',
    when(col('CO_GT_') < 2, 'Good')
    .when(col('CO_GT_') <= 5, 'Moderate')
    .when(col('CO_GT_') <= 10, 'Unhealthy')
    .otherwise('Danger')
)

In [43]:
df3.show()

+-----------+-------------+------------+------------+-----------+----+----+------+--------+
|PT08_S1_CO_|PT08_S2_NMHC_|PT08_S3_NOx_|PT08_S4_NO2_|PT08_S5_O3_|   T|  RH|CO_GT_|Category|
+-----------+-------------+------------+------------+-----------+----+----+------+--------+
|     1360.0|       1046.0|      1056.0|      1692.0|     1268.0|13.6|48.9|   2.6|Moderate|
|     1292.0|        955.0|      1174.0|      1559.0|      972.0|13.3|47.7|   2.0|Moderate|
|     1402.0|        939.0|      1140.0|      1555.0|     1074.0|11.9|54.0|   2.2|Moderate|
|     1376.0|        948.0|      1092.0|      1584.0|     1203.0|11.0|60.0|   2.2|Moderate|
|     1272.0|        836.0|      1205.0|      1490.0|     1110.0|11.2|59.6|   1.6|    Good|
|     1197.0|        750.0|      1337.0|      1393.0|      949.0|11.2|59.2|   1.2|    Good|
|     1185.0|        690.0|      1462.0|      1333.0|      733.0|11.3|56.8|   1.2|    Good|
|     1136.0|        672.0|      1453.0|      1333.0|      730.0|10.7|60.0|   1.

In [44]:
df3 = assembler.transform(df3)

In [45]:
df3.show(2)

+-----------+-------------+------------+------------+-----------+----+----+------+--------+--------------------+
|PT08_S1_CO_|PT08_S2_NMHC_|PT08_S3_NOx_|PT08_S4_NO2_|PT08_S5_O3_|   T|  RH|CO_GT_|Category|   feature_assembled|
+-----------+-------------+------------+------------+-----------+----+----+------+--------+--------------------+
|     1360.0|       1046.0|      1056.0|      1692.0|     1268.0|13.6|48.9|   2.6|Moderate|[1360.0,1046.0,10...|
|     1292.0|        955.0|      1174.0|      1559.0|      972.0|13.3|47.7|   2.0|Moderate|[1292.0,955.0,117...|
+-----------+-------------+------------+------------+-----------+----+----+------+--------+--------------------+
only showing top 2 rows



In [46]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [47]:
indexer = StringIndexer(inputCol='Category',outputCol='label')

In [48]:
df4 = indexer.fit(df3).transform(df3)

In [49]:
df4.show(2)

+-----------+-------------+------------+------------+-----------+----+----+------+--------+--------------------+-----+
|PT08_S1_CO_|PT08_S2_NMHC_|PT08_S3_NOx_|PT08_S4_NO2_|PT08_S5_O3_|   T|  RH|CO_GT_|Category|   feature_assembled|label|
+-----------+-------------+------------+------------+-----------+----+----+------+--------+--------------------+-----+
|     1360.0|       1046.0|      1056.0|      1692.0|     1268.0|13.6|48.9|   2.6|Moderate|[1360.0,1046.0,10...|  1.0|
|     1292.0|        955.0|      1174.0|      1559.0|      972.0|13.3|47.7|   2.0|Moderate|[1292.0,955.0,117...|  1.0|
+-----------+-------------+------------+------------+-----------+----+----+------+--------+--------------------+-----+
only showing top 2 rows



In [50]:
df4 = df4.select('feature_assembled','label')
df4.show(2)

+--------------------+-----+
|   feature_assembled|label|
+--------------------+-----+
|[1360.0,1046.0,10...|  1.0|
|[1292.0,955.0,117...|  1.0|
+--------------------+-----+
only showing top 2 rows



In [51]:
train,test = df4.randomSplit([0.8,0.2],seed = 42)

In [52]:
rf = RandomForestClassifier(featuresCol='feature_assembled',labelCol='label')

In [53]:
model = rf.fit(train)

In [54]:
pred = model.transform(test)

In [55]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acc = evaluator.evaluate(pred)
print("Classification Accuracy:", acc)

Classification Accuracy: 0.845
